In [4]:
%load_ext pycodestyle_magic

The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


In [5]:
%flake8_on

In [6]:
import numpy as np
import random as rd
import math
import IOHexperimenter as IOH
from IOHexperimenter import IOH_function
from shutil import make_archive
import datetime
np.random.seed()

In [8]:
class Simple_EA:
    """ A simple evolutionary algorithm object.

    Parameters:
    -----------------
    budget: int
        The budget for function evaluations.

    Attributes:
    -----------------
    d : int
        The number of dimensions, problem variables.

    mu : int
        The population size, number of parents.

    lamda : int
        The number of offspring individuals per iteration.

    pc : float
        The cross-over probability, must be 0>= pc >=1

    pop : array-like
        Matrix holding the current population.

    sigma : array-like, float
        Array containing the individual step-sizes.

    global_tau : float
        Constant for step size update in individual stepsize mutation.

    local_tau : float
        Constant for step size update in individual stepsize mutation.

    tau : float
         Constant for step size update in single stepsize mutation.

    """

    def __init__(self, budget):
        self.budget = budget
        self.d = 1
        self.mu = 15
        self.lamda = 100
        self.pc = 0.75
        self.pop = None
        self.sigma = None
        self.global_tau = 1.0
        self.local_tau = 1.0
        self.tau = 1.0

    def initialize_population(self):
        """ Creates a random population.

        Parameters:
        -------------
        None

        Returns:
        -------------
        population : array-like
            An array of mu individuals

        """
        population = np.zeros((self.mu, self.d))
        for i in range(0, self.d):
            for k in range(0, self.mu):
                population[k][i] = rd.uniform(-5, 5)
            return population

    def global_discrete_recombine(self):
        offspring = np.zeros((self.d))
        for i in range(0, self.d):
            random_parent = self.pop[rd.randrange(0, len(self.pop))]
            offspring[i] = random_parent[i]

        return offspring

    def update_sigma(self):
        new_sigma = self.sigma * math.exp(self.tau * np.random.normal(0, 1))
        return new_sigma

    def singlestep_mutate(self, ind, func):
        mutant = np.zeros(len(ind))
        for j in range(0, len(ind)):
            mutant[j] = ind[j] + self.sigma * np.random.normal(0, 1)

        mutant = np.clip(mutant, func.lowerbound, func.upperbound)

        return mutant

    def update_sigma_vec(self):
        updated_sigma = np.zeros(self.d)
        for i in range(0, self.d):
            updated_sigma[i] = self.sigma[i] * math.exp(
                self.global_tau * np.random.normal(0, 1)
                + self.local_tau * np.random.normal(0, 1))

        return updated_sigma

    def indstep_mutate(self, ind, func):
        mutant = np.zeros(len(ind))
        for j in range(0, len(ind)):
            mutant[j] = ind[j] + self.sigma[j] * np.random.normal(0, 1)
        mutant = np.clip(mutant, func.lowerbound, func.upperbound)

        return mutant

    def __call__(self, func):
        """ Runs the evolutionary algorithm.

        Parameters:
        --------------
        func : object-like
            The function to be optimized.

        Returns:
        --------------
        x_opt : array
                The best found solution.

        f_opt: float
               The fitness value for the best found solution.

        """

        # Initialize parameters depending on problem
        self.d = func.number_of_variables
        eval_budget = self.budget * self.d
        self.global_tau = 1 / math.sqrt(2*self.d)
        self.local_tau = 1 / math.sqrt(2*math.sqrt(self.d))

        """ For single step mutation, use:

        sigma = 10/6
        self.tau = 1 / math.sqrt(self.d)

        """

        # Initialization
        self.pop = self.initialize_population()    # parent population
        f = np.zeros((self.mu, 1))    # parents fitness
        xo = np.zeros((self.lamda, self.d))    # offspring population
        fo = np.zeros((self.lamda, 1))    # offspring fitness
        x_opt = None
        f_opt = np.inf
        self.sigma = np.zeros(self.d)
        for s in range(0, self.d):
            self.sigma[s] = 10 / 6

        # Evaluate initial parent population
        for i in range(0, self.mu):
            f[i] = func(self.pop[i])

        # Evaluation loop
        while func.evaluations < eval_budget and not func.best_so_far_precision <= 1e-8:
            # Update sigma
            self.sigma = self.update_sigma_vec()

            # Create new offspring generation
            for i in range(0, self.lamda):
                offspring = self.global_discrete_recombine()
                xo[i] = self.indstep_mutate(offspring, func)
                fo[i] = func(xo[i])

            # Selection
            # Create joint matrix of individuals and their fitness values
            m = np.concatenate((xo, fo), axis=1)
            # Sort based on fitness (lowest values highest ranking)
            sorted_m = m[np.argsort(m[:, self.d])]
            # Copy best individuals and remove fitness values
            xpnew = sorted_m[0:self.mu, 0:self.d]
            # Set new population for next generation
            self.pop = xpnew
            # Copy fitness values of new generation
            f = sorted_m[0:self.mu, self.d]

            # Check if new best individual is better than previous best
            if f[0] < f_opt:
                x_opt = self.pop[0]
                f_opt = f[0]

        return x_opt, f_opt

156:80: E501 line too long (88 > 79 characters)


In [26]:
fIDs = []
for fid in range(1, 25):
    fIDs.append(fid)

In [27]:
from IOHexperimenter import IOHexperimenter
exp = IOHexperimenter()
exp.initialize_BBOB(fIDs,[1],[2, 5, 10], 10)
path = "/Users/dschroeder/Documents/Master Computer Science/Master_Thesis/code_and_data/experiments/"
now = datetime.datetime.now()
experiment_ID = str(now.day) + "-" + str(now.month) + "-" + str(now.strftime('%H')) + str(now.strftime('%M')) + str(now.strftime('%S'))
experiment_name =  "Simple_EA/experiment" + experiment_ID
exp.set_logger_location(path + experiment_name, "run")
exp([Simple_EA(10000)])
#create a ZIP file for IOHprofiler upload
make_archive(path + experiment_name,"zip", path + experiment_name)

Running single-threaded
1 20015 1.3106249747139369e-08
1 20015 3.882610441619503e-06
1 20015 3.427183473873543e-08
1 20015 0.000842676516500117
1 20015 0.008483051865497575
1 20015 1.1099046853992764e-05
1 20015 1.7275467457025623e-05
1 20015 1.3580035735999449e-06
1 20015 0.0014570380148847331
1 20015 5.0772440158149486e-06
1 50015 0.13722414994430693
1 50015 0.30124664499809684
1 50015 1.1229338502632753
1 50015 1.1635924402806863
1 50015 0.8296043078087117
1 50015 0.8015891540780331
1 50015 0.01879345874614466
1 50015 1.5264110306929008
1 50015 0.016795017047622283
1 50015 0.06512614573774655
1 100015 21.981174389136278
1 100015 2.452107729045641
1 100015 6.211822012071952
1 100015 17.527538019772148
1 100015 1.825040510089943
1 100015 3.016461744977859
1 100015 4.871557266309146
1 100015 3.7872504207816458
1 100015 5.942759813620851
1 100015 11.11053249745699
2 20015 0.9777097214898081
2 20015 4.3041123539087485
2 20015 16.492830798661117
2 20015 0.0304210227768681
2 20015 0.618766

NameError: name 'shutil' is not defined

3:25: E231 missing whitespace after ','
3:29: E231 missing whitespace after ','
4:80: E501 line too long (101 > 79 characters)
6:80: E501 line too long (135 > 79 characters)
7:18: E222 multiple spaces after operator
10:1: E265 block comment should start with '# '
11:43: E231 missing whitespace after ','


In [28]:
make_archive(path + experiment_name,"zip", path + experiment_name)

'/Users/dschroeder/Documents/Master Computer Science/Master_Thesis/code_and_data/experiments/Simple_EA/experiment1-10-155352.zip'

1:36: E231 missing whitespace after ','
